# Projeto Final - Sistema de Controle Financeiro

## Grupo 1
* José Paulo Marchezi
* Renato Freitas
* Nicolas Evilasio
* Mayra Juliana Pereira Alves
* Rafael Gustavo Leite
* Alexandre Sá Freire Brandão


# Fin-Ada

O aplicativo utiliza dois arquivos como base de dados:

- investimentos.csv

- movimentacoes.csv



Importar modulos

In [1]:
from src.utils.funcoes import *

c:\Users\nicol\OneDrive - COMERCIO DE PERFUMARIA GINSENG LTDA\Python\SantanderCoders\Projeto\Santander-Coders-2023-ED\projeto_LPII


Parametros "fixos" de configuração:
* PATH: informa o diretório onde os dados serão armazenados
* TAXA_INVESTIMENTO: Valor utilizado no calculo do rentimento.

In [3]:
PATH = "./database"
TAXA_INVESTIMENTO = 0.01

## Iniciar Base de dados

Criamos uma função para incluir e gerar a nosa base de dados. Essa base é constuiodad e dois arquivos: 

* "investimentos.csv"
  - Armazena as entradas de investimentos
* "movimentacoes.csv"
  - Armazena as entradas de receita e despesas da conta 

Essa função oferece ao usuário duas opções para inclusão manual dos dados:

1. usuário informa a data (no formato ano-mes-dia); o valor; e tipo (investimento, receita, ou despesa).

2. a entradas serão lidas de um arquivo .csv, o usuario deve informar o nome desse arquivo.


In [ ]:
# nome csv : financas_aleatorias_20_linhas.csv
incluir_registros_base_dados(taxa=TAXA_INVESTIMENTO, path=PATH)

## Agrupar dados

In [ ]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
print(agrupar_movimentacoes(movimentacoes, agrupar_por="tipo"))
print(agrupar_movimentacoes(movimentacoes, agrupar_por="mes"))

investimentos = read_csv(f"{PATH}/investimentos.csv")
print(agrupar_movimentacoes(investimentos, agrupar_por="tipo"))
print(agrupar_movimentacoes(investimentos, agrupar_por="mes"))

## Atualizar registro

In [ ]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
movimentacoes[3]

In [ ]:
atualizar_registro(indice=4,
                       tipo="despesa",
                       database_path=PATH,
                       valor=13)

In [ ]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
movimentacoes[3]

In [ ]:
investimentos = read_csv(f"{PATH}/investimentos.csv")
investimentos[3]

In [ ]:

atualizar_registro(indice=4,
                       tipo="investimento",
                       database_path=PATH,
                       valor=13,
                       taxa=TAXA_INVESTIMENTO)

In [ ]:
investimentos = read_csv(f"{PATH}/investimentos.csv")
investimentos[3]

## Exportar os dados

In [ ]:
movimentacoes = read_csv(f"{PATH}/movimentacoes.csv")
exportar_relatorio_json(movimentacoes, formato='json', nome_arquivo='relatorio_movimentacoes')
investimentos = read_csv(f"{PATH}/investimentos.csv")
exportar_relatorio_json(investimentos, formato='json', nome_arquivo='relatorio_investimentos')

## Detelar registro

In [ ]:
deletar_registro(indice=3, tipo="despesa", database_path=PATH)

In [5]:
read_csv(os.path.join(PATH, 'movimentacoes.csv'))

[{'id': '0000001',
  'tipo': 'despesa',
  'valor': '-1000.0',
  'ano': '2023',
  'mes': '2',
  'dia': '12'},
 {'id': '0000002',
  'tipo': 'receita',
  'valor': '4787.0',
  'ano': '2023',
  'mes': '1',
  'dia': '18'},
 {'id': '0000003',
  'tipo': 'despesa',
  'valor': '-101.0',
  'ano': '2023',
  'mes': '2',
  'dia': '23'},
 {'id': '0000005',
  'tipo': 'receita',
  'valor': '3674.0',
  'ano': '2023',
  'mes': '4',
  'dia': '9'},
 {'id': '0000006',
  'tipo': 'receita',
  'valor': '5423.0',
  'ano': '2023',
  'mes': '6',
  'dia': '14'},
 {'id': '0000007',
  'tipo': 'receita',
  'valor': '2340.0',
  'ano': '2023',
  'mes': '6',
  'dia': '21'},
 {'id': '0000008',
  'tipo': 'despesa',
  'valor': '-861.0',
  'ano': '2023',
  'mes': '7',
  'dia': '26'},
 {'id': '0000009',
  'tipo': 'despesa',
  'valor': '-5815.0',
  'ano': '2023',
  'mes': '8',
  'dia': '9'},
 {'id': '0000010',
  'tipo': 'receita',
  'valor': '1040.0',
  'ano': '2023',
  'mes': '9',
  'dia': '4'},
 {'id': '0000011',
  'tipo': 